In [86]:
import confluent_kafka
import random
import json

In [91]:
actions = [
    'login',
    'logout',
    'purchase',
    'click'
]

group = 'group'
topic = 'example-topic'


def generate_data():
    for i in range(random.randint(150, 200)):
        id = random.randint(1, 100)
        yield {'id': id, 'action': actions[random.randint(0, len(actions) - 1)]}

In [103]:
config = {
    'bootstrap.servers': 'localhost:9092',
}

producer = confluent_kafka.Producer(config)

for i in generate_data():
    producer.produce(topic, value=json.dumps(i))
    producer.flush()



In [104]:
config = {
    'bootstrap.servers': 'localhost:9092', 
    'group.id': group,
    'auto.offset.reset': 'earliest'
}

consumer = confluent_kafka.Consumer(config)
consumer.subscribe([topic])
clicking_users = {}
purchasing_users = {}
count = 0
while True:
    msg = consumer.poll(1.0)
    if msg is not None:
        value = json.loads(msg.value().decode('utf-8'))
        user = value['id']
        op = value['action']
        if op == 'click':
            if user not in clicking_users:
                clicking_users[user] = 0
            clicking_users[user] += 1
        elif op == 'purchase':
            if user not in purchasing_users:
                purchasing_users[user] = 0
            purchasing_users[user] += 1
        count += 1
    if count >= 100:
        break

most_clicks= sorted([(k, v) for k, v in clicking_users.items()], key = lambda x: x[1])
most_purchases = sorted([(k, v) for k, v in purchasing_users.items()], key = lambda x: x[1])

most_clicks.reverse()
most_clicks = most_clicks[0]
most_purchases.reverse()
most_purchases = most_purchases[0]

In [105]:
print(most_clicks, most_purchases)

(84, 2) (3, 2)
